In [1]:
import numpy as np

In [2]:
import xgboost as xgb

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [18]:
features = pd.read_csv("features_11.csv")
kaggle = pd.read_csv('trocafone_kaggle_test.csv')
labels = pd.read_csv('labels_training_set.csv')

In [6]:
training = labels.set_index('person').join(features.set_index('person'))
train_labels = training['label']
train_data = training.drop('label',axis=1)

In [7]:
predict_data = kaggle.set_index('person').join(features.set_index('person'))

In [8]:
encoder = category_encoders.TargetEncoder(smoothing=True)
train_data_enc = encoder.fit_transform(np.array(train_data),np.array(train_labels))
predict_data_enc = encoder.transform(np.array(predict_data))

In [ ]:
xg_classifier = XGBClassifier()

In [10]:
xtrain,xtest,ytrain,ytest = train_test_split(train_data_enc,np.array(train_labels))

In [11]:
train = xgb.DMatrix(data=xtrain,label=ytrain)
test = xgb.DMatrix(data=xtest,label=ytest)
data = xgb.DMatrix(data=train_data_enc,label=np.array(train_labels))
pred = xgb.DMatrix(data=predict_data_enc)
params = {'colsample_bytree': 0.9,
 'gamma': 2,
 'learning_rate': 0.016238752458245277,
 'max_depth': 13,
 'min_child_weight': 5,
 'n_estimators': 100,
 'subsample': 0.7847190225361189,
 'eval_metric':'auc'}

In [12]:
bst = xgb.train(params,train,200,[(train,'train'),(test,'eval')])

[12:39:49] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 328 pruned nodes, max_depth=9
[0]	train-auc:0.755885	eval-auc:0.719793
[12:39:49] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 416 pruned nodes, max_depth=8
[1]	train-auc:0.762227	eval-auc:0.727012
[12:39:49] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 384 pruned nodes, max_depth=11
[2]	train-auc:0.778275	eval-auc:0.749767
[12:39:49] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 374 pruned nodes, max_depth=10
[3]	train-auc:0.781017	eval-auc:0.749003
[12:39:49] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 472 pruned nodes, max_depth=10
[4]	train-auc:0.780422	eval-auc:0.749224
[12:39:49] C:\Users\Adminis

In [13]:
bst = xgb.train(params,data,200)

[12:40:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 420 pruned nodes, max_depth=10
[12:40:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 532 pruned nodes, max_depth=8
[12:40:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 426 pruned nodes, max_depth=12
[12:40:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 66 extra nodes, 454 pruned nodes, max_depth=13
[12:40:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 498 pruned nodes, max_depth=9
[12:40:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 572 pruned nodes, max_depth=10
[12:40:08] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74

In [14]:
xg_prediction = bst.predict(pred)

In [15]:
submit = kaggle.set_index('person')

In [16]:
submit['label'] = xg_prediction

In [17]:
submit.to_csv('submit.csv')

In [97]:
bst.save_model('86605_xgboost_combinar_con_86602_catboost')

## Combinando con catboost (0.86605)

In [15]:
import catboost as cb

In [16]:
cb_classifier = cb.CatBoostClassifier(iterations=400,
                                     depth=8,
                                     l2_leaf_reg=100,
                                     random_strength=0.1,
                                     bagging_temperature=0,
                                     border_count=254,
                                     learning_rate=0.03)

In [17]:
cb_classifier.load_model('preentrenados/86602catboost')

In [18]:
cb_prediction = cb_classifier.predict_proba(predict_data)[:,1]

In [19]:
prediction = 0.9*cb_prediction+0.1*xg_prediction

In [20]:
submit = kaggle.set_index('person')

In [21]:
submit['label'] = prediction

In [22]:
submit.to_csv('submit.csv')